In [1]:
#Andrianna Anastasopoulou
#1115201300009
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud
import nltk 
from nltk.corpus import stopwords
import numpy as np
import re
import unicodedata
from sklearn.metrics import accuracy_score
from bs4 import BeautifulSoup
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [2]:
path = os.getcwd()
test_res = pd.read_csv(path+"/impermium_verification_labels.csv")

test = pd.read_csv(path+"/impermium_verification_set.csv")

train =  pd.read_csv(path+"/train.csv")

In [3]:

test=test.astype({'Comment': str})
test['Comment']=test['Comment'].apply(lambda col: col.lower())  #convert letters to small
test['Comment'] = test['Comment'].replace(r'\\n',' ', regex=True)
test['Comment'] = test['Comment'].map(lambda x: x.replace(r'"',''))
test['Comment'] = test['Comment'].map(lambda x: x.replace(r'\xa0',' '))
test['Comment'] = test['Comment'].map(lambda x: x.replace(r'\r',' '))
test['Comment'] = test['Comment'].map(lambda x: x.replace(r'_',' '))
test['Comment'] = [BeautifulSoup(text).get_text() for text in test['Comment'] ] #removes html
test['Comment'] = test['Comment'].map(lambda x: re.sub(r'http\S+', ' ', x)) #removes links
test['Comment'] = test['Comment'].map(lambda x: re.sub(r'@\S+', ' ', x))
test['Comment'] = test['Comment'].map(lambda x: re.sub(r'\\u\S+', ' ', x))

In [4]:
train=train.astype({'Comment': str})
train['Comment']=train['Comment'].apply(lambda col: col.lower())
train['Comment'] = train['Comment'].replace(r'\\n',' ', regex=True)
train['Comment'] = train['Comment'].map(lambda x: x.replace(r'"',''))
train['Comment'] = train['Comment'].map(lambda x: x.replace(r'\xa0',' '))
train['Comment'] = train['Comment'].map(lambda x: x.replace(r'\r',' '))
train['Comment'] = train['Comment'].map(lambda x: x.replace(r'_',' '))
train['Comment'] = train['Comment'].map(lambda x: re.sub(r'http\S+', ' ', x))
train['Comment'] = [BeautifulSoup(text).get_text() for text in train['Comment'] ]
train['Comment'] = train['Comment'].map(lambda x: re.sub(r'@\S+', ' ', x))
train['Comment'] = train['Comment'].map(lambda x: re.sub(r'\\u\S+', ' ', x))

In [5]:
dftrain=train.copy()
dftest=test.copy()

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
train_counts = count_vect.fit_transform(train['Comment'])
test_counts = count_vect.transform(test['Comment'])

In [7]:
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import f1_score

clf = ComplementNB()
clf.fit(train_counts,train['Insult'])
pred = clf.predict(test_counts)
print ('Test data accuracy:', accuracy_score(test_res['Insult'], pred))
print ('F1:',f1_score(test_res['Insult'], pred, average='binary'))


Test data accuracy: 0.6116331096196868
F1: 0.6337552742616033


In [8]:
dftrainA=train.copy()
dftestA=test.copy()

In [9]:
#LEMMATIZATION

In [10]:
dfLemmTrain=train.copy()
dfLemmTest=test.copy()

In [11]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer:
     def __init__(self):
         self.wnl = WordNetLemmatizer()
     def __call__(self, doc):
         return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [12]:
count_vect = CountVectorizer(tokenizer=LemmaTokenizer()) 
train_counts = count_vect.fit_transform(dfLemmTrain['Comment'])
test_counts = count_vect.transform(dfLemmTest['Comment'])

In [13]:
clf = ComplementNB() 
clf.fit(train_counts,dfLemmTrain['Insult'])
pred = clf.predict(test_counts)

print ('Test data accuracy:', accuracy_score(test_res['Insult'], pred.astype(int)))
print ('F1:',f1_score(test_res['Insult'], pred, average='binary'))

Test data accuracy: 0.6196868008948546
F1: 0.6284965034965034


In [14]:
#STOPWORDS

In [15]:
dftrainStop=train.copy()
dftestStop=test.copy()

In [16]:
stopwords = nltk.corpus.stopwords.words('english')

In [17]:
for i in stopwords :
    dftestStop = dftestStop.replace(to_replace=r'\b%s\b'%i, value="",regex=True)
for i in stopwords :
    dftrainStop = dftrainStop.replace(to_replace=r'\b%s\b'%i, value="",regex=True)

In [18]:
count_vect = CountVectorizer(tokenizer=LemmaTokenizer()) 
train_counts = count_vect.fit_transform(dftrainStop['Comment'])
test_counts = count_vect.transform(dftestStop['Comment'])

In [19]:
clf = ComplementNB() 
clf.fit(train_counts,dftrainStop['Insult'])
pred = clf.predict(test_counts)

print ('Test data accuracy:', accuracy_score(test_res['Insult'], pred.astype(int)))
print ('F1:',f1_score(test_res['Insult'], pred, average='binary'))

Test data accuracy: 0.6635346756152125
F1: 0.6217303822937627


In [20]:
#BIGRAMS

In [21]:
dfBigramTrain=train.copy()
dfBigramTest=test.copy()

In [22]:
count_vect = CountVectorizer(ngram_range=(2,2)) 
train_counts = count_vect.fit_transform(dfBigramTrain['Comment'])
test_counts = count_vect.transform(dfBigramTest['Comment'])

In [23]:
clf = ComplementNB() 
clf.fit(train_counts,dfBigramTrain['Insult'])
pred = clf.predict(test_counts)

print ('Test data accuracy:', accuracy_score(test_res['Insult'], pred.astype(int)))
print ('F1:',f1_score(test_res['Insult'], pred, average='binary'))

Test data accuracy: 0.6263982102908278
F1: 0.5924841386041971


In [24]:
#Laplace Smoothing

In [25]:
dfLSTrain=train.copy()
dfLSTest=test.copy()

In [26]:
count_vect = CountVectorizer() 
train_counts = count_vect.fit_transform(dfLSTrain['Comment'])
test_counts = count_vect.transform(dfLSTest['Comment'])

In [27]:

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB() #default laplace smoothing
clf.fit(train_counts,dfLSTrain['Insult'])
pred = clf.predict(test_counts)
print ('Test data accuracy:', accuracy_score(test_res['Insult'], pred.astype(int)))
print ('F1:',f1_score(test_res['Insult'], pred, average='binary'))


Test data accuracy: 0.680089485458613
F1: 0.6372399797057331


In [28]:
#Lemmatization,removed stopwords,use of bigrams and laplace smoothing together 

In [29]:
dfTrainCombined=train.copy()
dfTestCombined=test.copy()

In [30]:
count_vect = CountVectorizer(tokenizer=LemmaTokenizer(),ngram_range=(1,2),stop_words= 'english') 
train_counts = count_vect.fit_transform(dfTrainCombined['Comment'])
test_counts = count_vect.transform(dfTestCombined['Comment'])

In [31]:
clf = MultinomialNB() #default laplace smoothing
clf.fit(train_counts,dfTrainCombined['Insult'])
pred = clf.predict(test_counts)

print ('Test data accuracy:', accuracy_score(test_res['Insult'], pred.astype(int)))
print ('F1:',f1_score(test_res['Insult'], pred, average='binary'))


Test data accuracy: 0.6331096196868009
F1: 0.4429347826086957


In [32]:
#The Lemmatizer() runs before the removal of the stop words in  CountVectorizer() which worsens the scores

In [33]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

# nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [34]:
#lemmatization made scores worst for the parts of speech 

# dftrain['Comment'] = dftrain['Comment'].apply(lemmatize_text)  
# dftest['Comment'] = dftest['Comment'].apply(lemmatize_text)
# dftrain=dftrain.astype({'Comment': str})
# dftest=dftest.astype({'Comment': str})

In [35]:
# the dataframes should be tokenized in order to add the parts of speech
dftrain['Comment'] = dftrain['Comment'].apply(word_tokenize)
dftest['Comment'] = dftest['Comment'].apply(word_tokenize)

In [36]:
def countTags(text):
    posFractions=[]
    text = text.apply(lambda x: nltk.pos_tag(x))
    for x in text:
        total=0
        nouns=adjectives=adverbs=verbs=0
        for word in x:
            if (word[1] == "NN" or word[1] == "NNP" or word[1] == "NNS"):
                nouns+=1
            elif (word[1] == "JJ" or word[1] == "JJS" or word[1] == "JJR"):
                adjectives+=1
            elif (word[1] == "RB" or word[1] == "RBR" or word[1] == "RBS"):
                adverbs+=1
            elif (word[1] == "VB" or word[1] == "VBD" or word[1] == "VBG" or word[1] == "VBP" or word[1] == "VBZ"):
                verbs+=1
            total+=1
        if total>0:
            nouns=nouns/total
            adjectives=adjectives/total
            adverbs=adverbs/total
            verbs=verbs/total
        posFractions.append([nouns,adjectives,adverbs,verbs])
    return posFractions

In [37]:
dftrain=dftrain.astype({'Comment': str})
dftest=dftest.astype({'Comment': str})


In [38]:
dftrain['Comment'] = dftrain['Comment'].str.replace('\d+', '') # for digits
dftrain['Comment'] = dftrain['Comment'].str.replace(r'(\b\w{1,2}\b)', '') # for words
dftrain['Comment'] = dftrain['Comment'].str.replace('[^\w\s]', '') # for punctuation

dftest['Comment'] = dftest['Comment'].str.replace('\d+', '') # for digits
dftest['Comment'] = dftest['Comment'].str.replace(r'(\b\w{1,2}\b)', '') # for words
dftest['Comment'] = dftest['Comment'].str.replace('[^\w\s]', '') # for punctuation

In [39]:
tagsTrain=countTags(dftrain['Comment'])
tagsTest=countTags(dftest['Comment'])

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2),min_df=2)
tf_idf_train = vectorizer.fit_transform(dftrain['Comment'])
tf_idf_test = vectorizer.transform(dftest['Comment'])
tf_idf_train=np.append(tf_idf_train.toarray(),tagsTrain,axis=1)
tf_idf_test=np.append(tf_idf_test.toarray(),tagsTest,axis=1)

In [41]:
from sklearn import svm

clf1 = svm.SVC(gamma = 'scale' , C = 5. , kernel = 'rbf')
clf1.fit(tf_idf_train,dftrain['Insult'])

SVC(C=5.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [42]:
pred = clf1.predict(tf_idf_test)
print ('Test data accuracy:', accuracy_score(test_res['Insult'], pred))
print ('F1:',f1_score(test_res['Insult'], pred, average='macro'))

Test data accuracy: 0.7029082774049217
F1: 0.6891010654076857


In [43]:
from sklearn.ensemble import RandomForestClassifier

clf2 = RandomForestClassifier(n_estimators=50)
clf2.fit(tf_idf_train,dftrain['Insult'])
pred = clf2.predict(tf_idf_test)
print ('Test data accuracy:', accuracy_score(test_res['Insult'], pred))
print ('F1:',f1_score(test_res['Insult'], pred, average='micro'))


Test data accuracy: 0.6434004474272931
F1: 0.6434004474272931


In [44]:
#Score improvement

In [45]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

def stem_text(text):    
    stm = PorterStemmer()
    tokenized = word_tokenize(text)
    stem_tokes = []
    for toke in tokenized:
        stem_tokes.append(stm.stem(toke))
    stem_tokes
    combined = ''
    for stemmed in stem_tokes:
        combined += stemmed + ' '
    return combined

In [46]:
dftrainA['Comment']=dftrainA['Comment'].apply(stem_text)
dftestA['Comment']=dftestA['Comment'].apply(stem_text)

In [47]:
dftrainA['Comment'] = dftrainA['Comment'].apply(lemmatize_text)
dftestA['Comment'] = dftestA['Comment'].apply(lemmatize_text)


In [48]:
dftrainA=dftrainA.astype({'Comment': str})
dftestA=dftestA.astype({'Comment': str})

In [49]:
vectorizer = TfidfVectorizer(ngram_range=(1,2) # stop_words= 'english', removing stopwords drops the accuracy
                            )
tf_idf_train = vectorizer.fit_transform(dftrainA['Comment'])
tf_idf_test = vectorizer.transform(dftestA['Comment'])

In [50]:
clf1 = svm.SVC(gamma = 'scale' , C = 5. , kernel = 'linear',class_weight='balanced')
clf1.fit(tf_idf_train,dftrainA['Insult'])

pred = clf1.predict(tf_idf_test)
print ('Test data accuracy:', accuracy_score(test_res['Insult'], pred))
print ('F1:',f1_score(test_res['Insult'], pred, average='macro'))

Test data accuracy: 0.7230425055928411
F1: 0.7192430430481215


In [51]:
from sklearn.model_selection import cross_val_score
print("Support Vector Machines using tf-idf\n")
print ("10-fold cross validation:")
accuracy = cross_val_score(clf1, train_counts, dftrainA['Insult'], cv=10, scoring='accuracy')
print ('Accuracy', np.mean(accuracy))

f1s = cross_val_score(clf1, tf_idf_train, dftrainA['Insult'], cv=10, scoring='f1_weighted')
print ('F1', np.mean(f1s))

Support Vector Machines using tf-idf

10-fold cross validation:
Accuracy 0.8109924906711766
F1 0.8324991720786354
